In [ ]:
# pip install lxml - установка необходимой библиотеки

In [28]:
import requests
from lxml import html
import csv

# URL страницы с данными о коронавирусе
url = 'https://www.worldometers.info/coronavirus/'

# Задаем строку агента пользователя для имитации веб-браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def format_number(number_str):
    # Удаляем лишние символы, такие как пробелы и знаки плюса
    number_str = number_str.strip().replace('+', '').replace(' ', '')
    # Проверяем, является ли строка числом
    if number_str.isdigit():
        # Форматируем число с разделителями тысяч
        return "{:,}".format(int(number_str))
    return number_str

try:
    # Отправляем HTTP GET-запрос и получаем HTML-содержимое страницы
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Проверяем на наличие ошибок HTTP
    
    # Парсим HTML-содержимое страницы с помощью lxml
    tree = html.fromstring(response.content)
    
    # XPath выражения для выбора строк таблицы с данными по странам
    rows = tree.xpath('//table[@id="main_table_countries_today"]/tbody[1]/tr')
    
    # Извлекаем данные из таблицы
    extracted_data = []
    for row in rows:
        # Извлекаем текст каждой ячейки в строке, обрабатываем данные
        cell_data = []
        for cell in row.xpath('.//td'):
            text = cell.text_content().strip()
            formatted_text = format_number(text)
            cell_data.append(formatted_text)
        # Выбираем только нужные столбцы - 15 
        cell_data = cell_data[:15]  
        extracted_data.append(cell_data)
    
    # Сохраняем извлеченные данные в CSV-файл
    with open('covid_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Записываем заголовки столбцов
        writer.writerow(['#','Country Other', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'New Recovered', 'Active Cases', 'Serious/Critical', 'Total Cases/1M pop', 'Deaths/1M pop', 'Total Tests', 'Tests/1M pop', 'Population'])
        writer.writerows(extracted_data)
    
    print("Данные успешно сохранены в файл 'covid_data.csv'.")

except requests.HTTPError as e:
    # Обработка ошибок HTTP
    print(f"Ошибка HTTP: {e}")
except requests.RequestException as e:
    # Обработка других ошибок запроса
    print(f"Ошибка запроса: {e}")
except Exception as e:
    # Обработка всех остальных ошибок
    print(f"Произошла ошибка: {e}")

Данные успешно сохранены в файл 'covid_data.csv'.
